# Project Springfield - Player Detection Test
## Quick test of the PlayerDetector module

This notebook tests the basic functionality of our basketball player detection system.

In [ ]:
# Setup imports and paths
import sys
from pathlib import Path
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Add src to path
sys.path.append('../src')

from src.utils.config_manager import ConfigManager
from src.detection.player_detector import PlayerDetector
from src.utils.visualization import Visualizer

print("✅ Imports successful!")

In [ ]:
# Initialize components
print("⚙️ Loading configuration...")
config_manager = ConfigManager('../configs')

print("🤖 Initializing player detector...")
detector = PlayerDetector(config_manager)

print("🎨 Initializing visualizer...")
visualizer = Visualizer()

# Show model info
model_info = detector.get_model_info()
print("\n📋 Model Information:")
for key, value in model_info.items():
    print(f"   {key}: {value}")

In [ ]:
# Test with webcam (optional - comment out if no camera)
def test_webcam_frame():
    """Capture and test a single frame from webcam"""
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ No webcam available")
        return None
    
    ret, frame = cap.read()
    cap.release()
    
    if ret:
        print("📸 Captured webcam frame")
        return frame
    return None

# Uncomment to test with webcam:
# test_frame = test_webcam_frame()
# if test_frame is not None:
#     detections = detector.detect(test_frame)
#     print(f"🔍 Detected {len(detections)} players in webcam frame")

In [ ]:
# Create a test image with multiple people (simulating basketball players)
def create_test_image():
    """Create a simple test image for detection"""
    # Create a basketball court-like background
    img = np.ones((480, 640, 3), dtype=np.uint8) * 50  # Dark court
    
    # Add some court lines
    cv2.line(img, (50, 50), (590, 50), (255, 255, 255), 2)  # Top line
    cv2.line(img, (50, 430), (590, 430), (255, 255, 255), 2)  # Bottom line
    cv2.line(img, (50, 50), (50, 430), (255, 255, 255), 2)  # Left line
    cv2.line(img, (590, 50), (590, 430), (255, 255, 255), 2)  # Right line
    
    # Add center circle
    cv2.circle(img, (320, 240), 50, (255, 255, 255), 2)
    
    # Add some colored rectangles to simulate players
    # Player 1 (red jersey)
    cv2.rectangle(img, (150, 180), (190, 260), (0, 0, 255), -1)
    cv2.circle(img, (170, 170), 15, (255, 200, 200), -1)  # Head
    
    # Player 2 (blue jersey) 
    cv2.rectangle(img, (450, 200), (490, 280), (255, 0, 0), -1)
    cv2.circle(img, (470, 190), 15, (255, 200, 200), -1)  # Head
    
    # Player 3 (green jersey)
    cv2.rectangle(img, (300, 150), (340, 230), (0, 255, 0), -1)
    cv2.circle(img, (320, 140), 15, (255, 200, 200), -1)  # Head
    
    return img

# Create and display test image
test_image = create_test_image()
plt.figure(figsize=(10, 6))
plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
plt.title("Test Basketball Court Image")
plt.axis('off')
plt.show()

print("🏀 Created test basketball court image")

In [ ]:
# Test detection on our test image
print("🔍 Running player detection...")
detections = detector.detect(test_image)

print(f"✅ Detection complete! Found {len(detections)} potential players")

# Print detection details
for i, detection in enumerate(detections):
    bbox = detection['bbox']
    conf = detection['confidence']
    print(f"  Player {i+1}: confidence={conf:.3f}, bbox=[{bbox[0]:.0f}, {bbox[1]:.0f}, {bbox[2]:.0f}, {bbox[3]:.0f}]")

In [ ]:
# Visualize detection results
if len(detections) > 0:
    annotated_image = visualizer.draw_detections(
        test_image, 
        detections,
        class_names={0: "Player"},
        show_confidence=True
    )
    
    # Display results side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    ax1.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB))
    ax1.set_title("Original Image")
    ax1.axis('off')
    
    ax2.imshow(cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB))
    ax2.set_title(f"Detection Results ({len(detections)} players found)")
    ax2.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("🎯 Detection visualization complete!")
else:
    print("⚠️ No players detected in test image")
    print("💡 This might be because:")
    print("   - The test image is too simple/artificial")
    print("   - Detection threshold is too high")
    print("   - Model needs real basketball footage")

In [ ]:
# Performance test
import time

print("⚡ Running performance test...")
n_runs = 10
times = []

for i in range(n_runs):
    start_time = time.time()
    detections = detector.detect(test_image)
    end_time = time.time()
    
    times.append(end_time - start_time)
    if i % 5 == 0:
        print(f"  Run {i+1}/{n_runs}...")

avg_time = np.mean(times)
fps = 1.0 / avg_time

print(f"\n📊 Performance Results:")
print(f"   Average detection time: {avg_time*1000:.1f} ms")
print(f"   Estimated FPS: {fps:.1f}")
print(f"   Device: {detector.device}")

if fps < 10:
    print("⚠️ Low FPS detected. Consider:")
    print("   - Using GPU if available")
    print("   - Using smaller model (yolov8n instead of yolov8s)")
    print("   - Reducing input resolution")
else:
    print("✅ Good performance for real-time processing!")

## Next Steps

If this test works well, you can:

1. **Test with real basketball footage** - Download some NBA/college basketball videos
2. **Run the full demo script** - Process entire videos or use webcam
3. **Add ball detection** - Create `BallDetector` class
4. **Implement tracking** - Connect detections across frames
5. **Extract statistics** - Count possessions, shots, movement patterns

**To run this notebook:**
```bash
cd Project_Springfield
jupyter lab notebooks/
```